This notebook provides example code for creating a basic RAG system.
May 2025

In [ ]:
# !uv add langchain openai faiss-cpu sentence-transformers

Resolved 104 packages in 460ms                                       
Prepared 45 packages in 1.80s                                            
Uninstalled 1 package in 0.91ms
Installed 70 packages in 263ms                              
 + annotated-types==0.7.0
 + anyio==4.9.0
 + certifi==2025.4.26
 + charset-normalizer==3.4.2
 + distro==1.9.0
 + faiss-cpu==1.11.0
 + filelock==3.18.0
 + fsspec==2025.3.2
 + greenlet==3.2.2
 + h11==0.16.0
 + httpcore==1.0.9
 + httpx==0.28.1
 + huggingface-hub==0.31.4
 + idna==3.10
 + jinja2==3.1.6
 + jiter==0.10.0
 + joblib==1.5.0
 + jsonpatch==1.33
 + jsonpointer==3.0.0
 + langchain==0.3.25
 + langchain-core==0.3.60
 + langchain-text-splitters==0.3.8
 + langsmith==0.3.42
 + markupsafe==3.0.2
 + mpmath==1.3.0
 + networkx==3.4.2
 + numpy==2.2.6
 + nvidia-cublas-cu12==12.6.4.1
 + nvidia-cuda-cupti-cu12==12.6.80
 + nvidia-cuda-nvrtc-cu12==12.6.77
 + nvidia-cuda-runtime-cu12==12.6.77
 + nvidia-cudnn-cu12==9.5.1.17
 + nvidia-cufft-cu12==11.3.0.4
 + nvidia-cu

In [ ]:
# Step 1: Imports
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

import os

# Step 2: Set OpenAI API key (replace with your key or use environment variable)
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# Step 3: Load documents (replace 'data.txt' with your own file)
loader = TextLoader("data/data.txt")
documents = loader.load()

# Step 4: Split into chunks
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

# Step 5: Embed and index with FAISS
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embedding_model)

# Step 6: Set up retriever and LLM
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
llm = OpenAI(temperature=0)

# Step 7: Set up RAG pipeline
rag_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)


In [ ]:
# Step 8: Ask a question
query = "What is the main idea of the document?"
result = rag_chain(query)

# Step 9: Display result
print("Answer:\n", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print("-", doc.metadata.get("source", "Unknown"), "\n", doc.page_content[:200], "\n")